<a href="https://colab.research.google.com/github/HongYeongPyo/first-repository/blob/Dataton/Dataton01_250530.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 문제 분석 과정
1. 문제 정의 및 가설검정
2. 데이터 준비 및 전처리
3. 라이프스타일 변수 분석
4. 정신건강 지표분석
5. 가설별 정량적 검정
6. 예측 모델링(선택)
7. 시각화 및 해석
8. 결론 도출 및 보고서 작성


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [28]:
# [1] 라이브러리 불러오기
import os
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

# [2] Google Drive 마운트
drive.mount('/content/drive')

# [3] 실제 경로 설정 (공백 포함 주의, 오타 수정)
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Dataton/playground-series-s4e11'

# [4] 데이터 불러오기
train_df = pd.read_csv(
    os.path.join(DATA_PATH, 'train.csv'),
    index_col=0,
    parse_dates=True
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 140700 entries, 0 to 140699
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Name                                   140700 non-null  object 
 1   Gender                                 140700 non-null  object 
 2   Age                                    140700 non-null  float64
 3   City                                   140700 non-null  object 
 4   Working Professional or Student        140700 non-null  object 
 5   Profession                             104070 non-null  object 
 6   Academic Pressure                      27897 non-null   float64
 7   Work Pressure                          112782 non-null  float64
 8   CGPA                                   27898 non-null   float64
 9   Study Satisfaction                     27897 non-null   float64
 10  Job Satisfaction                       112790 non-null  float

##  각 컬럼 설명
| 컬럼명                                    | 설명            | 예시 값                   | 데이터 타입  | 비고                            |
| -------------------------------------- | ------------- | ---------------------- | ------- | ----------------------------- |
| `Name`                                 | 응답자 이름 또는 식별자 | John                   | object  | 분석엔 불필요하면 제거 가능               |
| `Gender`                               | 성별            | Male / Female          | object  | 범주형                           |
| `Age`                                  | 나이            | 19.0                   | float64 | 수치형 (정수로 변환 가능)               |
| `City`                                 | 거주 도시         | Seoul, Mumbai          | object  | 범주형                           |
| `Working Professional or Student`      | 직업 상태         | Student / Professional | object  | 범주형                           |
| `Profession`                           | 직업 (선택 응답)    | Engineer / None        | object  | 결측 있음                         |
| `Academic Pressure`                    | 학업 스트레스 정도    | 1.0 \~ 10.0            | float64 | 일부 결측 있음                      |
| `Work Pressure`                        | 업무 스트레스 정도    | 1.0 \~ 10.0            | float64 | 일부 결측                         |
| `CGPA`                                 | 학점 (평균 성적)    | 6.8                    | float64 | 일부 결측                         |
| `Study Satisfaction`                   | 학업 만족도        | 1.0 \~ 10.0            | float64 | 일부 결측                         |
| `Job Satisfaction`                     | 직업 만족도        | 1.0 \~ 10.0            | float64 | 일부 결측                         |
| `Sleep Duration`                       | 하루 평균 수면 시간   | 7 hours                | object  | 수치로 변환 필요                     |
| `Dietary Habits`                       | 식습관           | Balanced / Junk        | object  | 범주형                           |
| `Degree`                               | 학위 수준         | Bachelor / None        | object  | 범주형                           |
| `Have you ever had suicidal thoughts?` | 자살 충동 경험 유무   | Yes / No               | object  | 민감 변수                         |
| `Work/Study Hours`                     | 일/공부 시간       | 4.0                    | float64 | 연속형 변수                        |
| `Financial Stress`                     | 경제적 스트레스 정도   | 1.0 \~ 10.0            | float64 | 일부 결측                         |
| `Family History of Mental Illness`     | 정신질환 가족력 여부   | Yes / No               | object  | 범주형                           |
| `Depression`                           | 우울증 여부 (라벨)   | 0 / 1                  | int64   | 종속 변수 (binary classification) |


In [33]:
train_df.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [35]:
train_df.shape


(140700, 19)

In [36]:
# 결측치 개수 및 비율 계산
missing_df = train_df.isnull().sum().to_frame(name='MissingCount')
missing_df['MissingRatio (%)'] = (missing_df['MissingCount'] / len(train_df)) * 100

# 결측치가 있는 컬럼만 필터링
missing_df = missing_df[missing_df['MissingCount'] > 0]

# 결측치 많은 순으로 정렬
missing_df.sort_values(by='MissingCount', ascending=False, inplace=True)

# 결과 출력
print(missing_df)


                    MissingCount  MissingRatio (%)
Academic Pressure         112803         80.172708
Study Satisfaction        112803         80.172708
CGPA                      112802         80.171997
Profession                 36630         26.034115
Work Pressure              27918         19.842217
Job Satisfaction           27910         19.836532
Dietary Habits                 4          0.002843
Financial Stress               4          0.002843
Degree                         2          0.001421


### 분석 관점에서의 ‘대도시’ 정의 기준


In [37]:
# 1. city 컬럼에 존재하는 고유 도시 목록과 빈도 확인
city_counts = train_df['City'].value_counts()
print(city_counts.head(20))

City
Kalyan           6591
Patna            5924
Vasai-Virar      5765
Kolkata          5689
Ahmedabad        5613
Meerut           5528
Ludhiana         5226
Pune             5210
Rajkot           5207
Visakhapatnam    5176
Srinagar         5074
Mumbai           4966
Indore           4872
Agra             4684
Surat            4636
Varanasi         4606
Vadodara         4568
Hyderabad        4496
Kanpur           4398
Jaipur           4328
Name: count, dtype: int64


In [38]:
train_df['City'].unique()


array(['Ludhiana', 'Varanasi', 'Visakhapatnam', 'Mumbai', 'Kanpur',
       'Ahmedabad', 'Thane', 'Nashik', 'Bangalore', 'Patna', 'Rajkot',
       'Jaipur', 'Pune', 'Lucknow', 'Meerut', 'Agra', 'Surat',
       'Faridabad', 'Hyderabad', 'Srinagar', 'Ghaziabad', 'Kolkata',
       'Chennai', 'Kalyan', 'Nagpur', 'Vadodara', 'Vasai-Virar', 'Delhi',
       'Bhopal', 'Indore', 'Ishanabad', 'Vidhi', 'Ayush', 'Gurgaon',
       'Krishna', 'Aishwarya', 'Keshav', 'Harsha', 'Nalini', 'Aditya',
       'Malyansh', 'Raghavendra', 'Saanvi', 'M.Tech', 'Bhavna',
       'Less Delhi', 'Nandini', 'M.Com', 'Plata', 'Atharv', 'Pratyush',
       'City', '3.0', 'Less than 5 Kalyan', 'MCA', 'Mira', 'Moreadhyay',
       'Morena', 'Ishkarsh', 'Kashk', 'Mihir', 'Vidya', 'Tolkata', 'Anvi',
       'Krinda', 'Ayansh', 'Shrey', 'Ivaan', 'Vaanya', 'Gaurav', 'Harsh',
       'Reyansh', 'Kashish', 'Kibara', 'Vaishnavi', 'Chhavi', 'Parth',
       'Mahi', 'Tushar', 'MSc', 'No', 'Rashi', 'ME', 'Molkata',
       'Researcher', '

In [39]:
import pandas as pd

# [1] train 데이터 로드
# traib_df = pd.read_csv('train.csv')

# [2] 외부 도시 통계 데이터 로드
city_info_df = pd.read_csv('city_population_gdp.csv')

# [3] 대도시 조건 필터링 (인구 ≥ 1,000,000 또는 GDP ≥ 50)
megacities = city_info_df[
    (city_info_df['Population'] >= 1_000_000) |
    (city_info_df['GDP (Billion USD)'] >= 50)
]['City'].tolist()

# [4] train 데이터에서 해당 도시만 필터링
filtered_train = train_df[train_df['City'].isin(megacities)]

# [5] 결과 확인
print(filtered_train['City'].value_counts())
print(f"✅ 추출된 도시 수: {filtered_train['City'].nunique()}")
print(f"✅ 추출된 데이터 수: {len(filtered_train)}")




City
Patna        5924
Kolkata      5689
Ahmedabad    5613
Pune         5210
Mumbai       4966
Indore       4872
Surat        4636
Hyderabad    4496
Kanpur       4398
Jaipur       4328
Lucknow      4280
Nagpur       4209
Bangalore    4123
Chennai      4044
Delhi        3593
Bhopal       3475
Name: count, dtype: int64
✅ 추출된 도시 수: 16
✅ 추출된 데이터 수: 73856


In [40]:
import pandas as pd

# [1] 데이터 불러오기
# df = pd.read_csv('train.csv')
city_info_df = pd.read_csv('city_population_gdp.csv')

# [2] 중소도시 조건 필터링: 인구 < 1,000,000 AND GDP < 50
small_cities = city_info_df[
    (city_info_df['Population'] < 1_000_000) &
    (city_info_df['GDP (Billion USD)'] < 50)
]['City'].tolist()

# [3] train.csv에서 중소도시 데이터 추출
filtered_small_city_df = train_df[train_df['City'].isin(small_cities)]

# [4] 결과 확인
print(filtered_small_city_df['City'].value_counts())
print(f"추출된 중소도시 수: {filtered_small_city_df['City'].nunique()}")
print(f" 추출된 데이터 수: {len(filtered_small_city_df)}")


Series([], Name: count, dtype: int64)
추출된 중소도시 수: 0
 추출된 데이터 수: 0


###  train 데이터 셋의 도시가 중소도시로 분류되지 않음:
-  train.csv 데이터에는 존재하지만, city_population_gdp.csv의 중소도시 목록에는 포함되지 않은 경우가 발생했을 가능성이 매우 높습니다.
- train.csv에는 중소도시 도시명이 존재하지만, 외부 파일에서 그 도시들이 중소도시로 분류되지 않았거나, 도시명이 일치하지 않아 매칭이 안 된 것입니다.

In [41]:
print(train_df['City'].unique())        # 고유 도시 목록
print(len(train_df['City'].unique()))   # 도시 개수


['Ludhiana' 'Varanasi' 'Visakhapatnam' 'Mumbai' 'Kanpur' 'Ahmedabad'
 'Thane' 'Nashik' 'Bangalore' 'Patna' 'Rajkot' 'Jaipur' 'Pune' 'Lucknow'
 'Meerut' 'Agra' 'Surat' 'Faridabad' 'Hyderabad' 'Srinagar' 'Ghaziabad'
 'Kolkata' 'Chennai' 'Kalyan' 'Nagpur' 'Vadodara' 'Vasai-Virar' 'Delhi'
 'Bhopal' 'Indore' 'Ishanabad' 'Vidhi' 'Ayush' 'Gurgaon' 'Krishna'
 'Aishwarya' 'Keshav' 'Harsha' 'Nalini' 'Aditya' 'Malyansh' 'Raghavendra'
 'Saanvi' 'M.Tech' 'Bhavna' 'Less Delhi' 'Nandini' 'M.Com' 'Plata'
 'Atharv' 'Pratyush' 'City' '3.0' 'Less than 5 Kalyan' 'MCA' 'Mira'
 'Moreadhyay' 'Morena' 'Ishkarsh' 'Kashk' 'Mihir' 'Vidya' 'Tolkata' 'Anvi'
 'Krinda' 'Ayansh' 'Shrey' 'Ivaan' 'Vaanya' 'Gaurav' 'Harsh' 'Reyansh'
 'Kashish' 'Kibara' 'Vaishnavi' 'Chhavi' 'Parth' 'Mahi' 'Tushar' 'MSc'
 'No' 'Rashi' 'ME' 'Molkata' 'Researcher' 'Kagan' 'Armaan' 'Ithal'
 'Nalyan' 'Dhruv' 'Galesabad' 'Itheg' 'Aaradhya' 'Pooja' 'Khushi'
 'Khaziabad' 'Jhanvi' 'Unirar']
98


In [42]:
# 도시명을 소문자로 정규화 후 비교
train_df['City_clean'] = train_df['City'].str.strip().str.lower()
city_info_df['City_clean'] = city_info_df['City'].str.strip().str.lower()

# 중소도시 추출 (정규화된 도시명 기준)
small_cities = city_info_df[
    (city_info_df['Population'] < 1_000_000) &
    (city_info_df['GDP (Billion USD)'] < 50)
]['City_clean'].tolist()

# train.csv에서 중소도시만 필터링
filtered_small_city_df = train_df[train_df['City_clean'].isin(small_cities)]


In [43]:
print("중소도시 후보 (외부 파일 기준):", small_cities)

print("train.csv에 있는 중소도시:", set(train_df['City_clean']) & set(small_cities))



중소도시 후보 (외부 파일 기준): []
train.csv에 있는 중소도시: set()


## 중소도시도 포함된 확장된 외부 파일을 작성해야 함

In [44]:
import pandas as pd

# 인구 및 GDP 기준으로 대도시 + 중소도시 포함
data = {
    'City': [
        # 대도시 (인구 ≥ 1,000,000 or GDP ≥ 50B)
        'Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Ahmedabad',
        'Chennai', 'Kolkata', 'Pune', 'Surat', 'Jaipur',

        # 중소도시 (인구 < 1,000,000 and GDP < 50B)
        'Bhopal', 'Patna', 'Indore', 'Nagpur', 'Kanpur',
        'Ludhiana', 'Vadodara', 'Varanasi', 'Ranchi', 'Raipur',
        'Gwalior', 'Amritsar', 'Jodhpur', 'Guwahati', 'Bhubaneswar',
        'Mysore', 'Noida'
    ],
    'Population': [
        # 대도시
        20411000, 16787941, 8443675, 6809970, 5570585,
        7090000, 4486679, 3124458, 4467797, 3073350,

        # 중소도시
        1798218, 1684222, 1964086, 2405665, 2767031,
        1610000, 2065771, 1201815, 1120000, 1070000,
        1050000, 1130000, 1140000, 957352, 881988,
        920550, 642381
    ],
    'GDP (Billion USD)': [
        # 대도시
        368, 293, 110, 74, 68,
        78, 150, 69, 59, 45,

        # 중소도시
        21, 20, 23, 28, 25,
        18, 20, 15, 16, 14,
        13, 12, 15, 13, 12,
        11, 10
    ]
}

# 데이터프레임 생성
city_info_df = pd.DataFrame(data)

# CSV 파일 저장
city_info_df.to_csv('city_population_gdp.csv', index=False)

print("✅ 'city_population_gdp.csv' 파일이 확장 버전으로 생성되었습니다.")


✅ 'city_population_gdp.csv' 파일이 확장 버전으로 생성되었습니다.


In [45]:
city_info_df = pd.read_csv('city_population_gdp.csv')

# 중소도시 필터링
small_cities = city_info_df[
    (city_info_df['Population'] < 1_000_000) &
    (city_info_df['GDP (Billion USD)'] < 50)
]['City'].tolist()
